# Jupyter执行LLaMA Demo

In [1]:
import os
import torch
os.environ['CUDA_VISIBLE_DEVICES']='6,7'
print(torch.cuda.device_count())
import transformers
from peft import PeftModel
from transformers import LlamaForCausalLM, LlamaConfig, LlamaTokenizer

2


In [2]:
!pwd

/home/jiw203/wjn/InstructGraph/examples/demo


### 加载LLaMA2与PEFT模型

In [48]:
model_name_or_path = "/home/jiw203/wjn/pre-trained-lm/Llama-2-7b-hf/"
peft_model = "/home/jiw203/wjn/InstructGraph/output/instruction_tuning/fsdp_peft_flash_1500k/llama2-peft"

In [4]:
# Function to load the main model for text generation
def load_model(model_name, quantization):
    model = LlamaForCausalLM.from_pretrained(
        model_name,
        return_dict=True,
        load_in_8bit=quantization,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
    return model


# Function to load the PeftModel for performance optimization
def load_peft_model(model, peft_model):
    peft_model = PeftModel.from_pretrained(model, peft_model)
    return peft_model

# Loading the model from config to load FSDP checkpoints into that
def load_llama_from_config(config_path):
    model_config = LlamaConfig.from_pretrained(config_path) 
    model = LlamaForCausalLM(config=model_config)
    return model

In [49]:
model = load_model(model_name_or_path, quantization=False)
model = load_peft_model(model, peft_model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
     

In [23]:
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

### 自定义输入

In [74]:
# instruction = (
#     'You are a good graph reasoner. Give you a graph language that describes a graph structure and node information. You need to understand the graph and the task definition, and answer the question.\nNote: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with a directed edge. \n```\nGraph[name="factual-graph"] {\n    entity_list = [\'Raid on Unadilla and Onaquaga\', \'New York\'];\n    triple_list = [("Raid on Unadilla and Onaquaga" -> "New York")[relation="located in the administrative territorial entity"], ("Raid on Unadilla and Onaquaga" -> "New York")[relation="location"]];\n}\n```\nTask definition: given an event title and a factual knowledge graph, generate an event narration.\nQ: Please generate an event narration based on the knowledge graph and the title "Raid on Unadilla and Onaquaga".\nA:'
# )
instruction = """You are a good graph generator. You need to understand the task definition and generate a graph language to answer the question. 
Note: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with a directed edge. 
Task definition: given a factual knowledge question, please think step by step: 1) find the topic entity and generate a corresponding knowledge subgraph to express the world knowledge information, 2) then generate a thinking explanation to describe this reasoning, and 3) finally output the final answer. Note that: 1) the graph is a directed-weighted graph and the name is "factual-knowledge-graph". 2) The generated graph language should be a code-like structure, and the skeleton format can be expressed as the following:
```
Graph[name="factual-knowledge-graph"] {
    entity_list = ["xxx", ...];
    triple_list = [("xxx" -> "xxx")[relation="xxx"], ...];
}
```
Q: What's the birth country of the film "Avatar"'s director?
A:"""

# instruction = "What's your name?"
batch = tokenizer(instruction, return_tensors="pt")
# print(batch)

### 模型生成

In [77]:
batch = {k: v.to("cuda") for k, v in batch.items()}
with torch.no_grad():
    outputs = model.generate(
        **batch,
        max_new_tokens=500,
        do_sample=False,
        top_p=1.0,
        temperature=1.0,
        min_length=None,
        use_cache=True,
        top_k=50,
        repetition_penalty=1.0,
        length_penalty=1
    )
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

You are a good graph generator. You need to understand the task definition and generate a graph language to answer the question. 
Note: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with a directed edge. 
Task definition: given a factual knowledge question, please think step by step: 1) find the topic entity and generate a corresponding knowledge subgraph to express the world knowledge information, 2) then generate a thinking explanation to describe this reasoning, and 3) finally output the final answer. Note that: 1) the graph is a directed-weighted graph and the name is "factual-knowledge-graph". 2) The generated graph language should be a code-like structure, and the skeleton format can be expressed as the following:
```
Graph[name="factual-knowledge-graph"] {
    entity_list = ["xxx", ...];
    triple_list = [("xxx" -> "xxx")[relation="xxx"], ...];
}
```
Q: What's the birth country of the film "Avat

## 模型生成能力初检

graph structure modeling
- Connectivity：可以生成，指令理解、内容连贯度和形式都符合预期，但准确率需要优化
- Cycle：可以生成，指令理解、内容连贯度和形式都符合预期，但准确率需要优化
- Topological Sort：可以生成，指令理解、内容连贯度和形式都符合预期，但准确率需要优化
- Shortest Path：可以生成，指令理解、内容连贯度和形式都符合预期，但准确率需要优化
- Maximum Flow：可以生成，指令理解、内容连贯度和形式都符合预期，但准确率需要优化
- Bipartite Graph Matching：可以生成，指令理解、内容连贯度和形式都符合预期，
- Hamilton Path：可以生成，指令理解、内容连贯度和形式都符合预期，但生成结果存在一定问题
- Graph Degree：训练数据存在错误：计算degree但是标签却为yes/no

graph caption generation:
- wikipedia：可以生成，指令理解、内容连贯度和形式都符合预期
- webnlg：可以生成，指令理解、内容连贯度和形式都符合预期
- agenda：可以生成，指令理解、内容连贯度和形式都符合预期
- genwiki：可以生成，指令理解、内容连贯度和形式都符合预期
- EventNarrative：可以生成，指令理解、内容连贯度和形式都符合预期
- XAlign：可以生成，指令理解、内容连贯度和形式都符合预期

graph question answering：
- pathquestions：可以生成，指令理解、内容连贯度和形式都符合预期
- wc2014：可以生成，指令理解、内容连贯度和形式都符合预期
- grailqa：可以生成，指令理解、内容连贯度和形式都符合预期
- webquestion：可以生成，指令理解、内容连贯度和形式都符合预期
- WikiTableQuestions：可以生成，指令理解、内容连贯度和形式都符合预期

graph node cls：
- cora：可以生成，指令理解、内容连贯度和形式都符合预期
- citeseer：可以生成，指令理解、内容连贯度和形式都符合预期
- pubmed：可以生成，指令理解、内容连贯度和形式都符合预期
- ogbn-arxiv：可以生成，指令理解、内容连贯度和形式都符合预期
- ogbn-product：可以生成，指令理解、内容连贯度和形式都符合预期

graph link prediction
- wikidata：可以生成，指令理解、内容连贯度和形式都符合预期
- FB15k-237：可以生成，指令理解、内容连贯度和形式都符合预期
- ConceptNet：可以生成，指令理解、内容连贯度和形式都符合预期

Graph Relevance Inspection：可以生成，指令理解、内容连贯度和形式都符合预期

Graph Collaboration Filtering：
- moivelens：可以生成，指令理解、内容连贯度和形式都符合预期
- Amazon：可以生成，指令理解、内容连贯度和形式都符合预期
- LastFM：可以生成，指令理解、内容连贯度和形式都符合预期

knowledge graph generation：（调整一下顺序，将passage放在q的前面）
- wikidata：可以生成，指令理解、内容连贯度和形式都符合预期题
- instructuie：可以生成，指令理解、内容连贯度和形式都符合预期
- instructkgc：可以生成，指令理解、内容连贯度和形式都符合预期

Structure Graph Generation：（调整一下顺序，将passage放在q的前面）
可以生成，指令理解、内容连贯度和形式都符合预期

Graph thought modeling: 没问题，但是graph language
probing：没问题，但是graph language

In [23]:
## 错误记录

In [24]:
"""
You are a good graph reasoner. Give you a graph language that describe a graph structure and node infotmation. You need to understanding the graph and the task definition, and answer the question.
Note: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with an directed edge. 
```
Graph[name="maximum-flow"] {
    node_list = [0, 1, 2, 3, 4, 5];
    edge_list = [(0 -> 4)[capacity=3], (0 -> 1)[capacity=6], (0 -> 2)[capacity=3], (1 -> 2)[capacity=4], (1 -> 0)[capacity=4], (2 -> 4)[capacity=9], (2 -> 3)[capacity=7], (3 -> 4)[capacity=4], (3 -> 0)[capacity=5], (4 -> 1)[capacity=4], (4 -> 2)[capacity=4], (4 -> 0)[capacity=2], (5 -> 0)[capacity=3]];
}
```
Task definition: calculate the maximum flow between two nodes in the graph.
Q: What is the maximum flow from node 2 to node 4?
A:

```
max_flow_from_node_2_to_node_4 = 12
```
"""

'\nYou are a good graph reasoner. Give you a graph language that describe a graph structure and node infotmation. You need to understanding the graph and the task definition, and answer the question.\nNote: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with an directed edge. \n```\nGraph[name="maximum-flow"] {\n    node_list = [0, 1, 2, 3, 4, 5];\n    edge_list = [(0 -> 4)[capacity=3], (0 -> 1)[capacity=6], (0 -> 2)[capacity=3], (1 -> 2)[capacity=4], (1 -> 0)[capacity=4], (2 -> 4)[capacity=9], (2 -> 3)[capacity=7], (3 -> 4)[capacity=4], (3 -> 0)[capacity=5], (4 -> 1)[capacity=4], (4 -> 2)[capacity=4], (4 -> 0)[capacity=2], (5 -> 0)[capacity=3]];\n}\n```\nTask definition: calculate the maximum flow between two nodes in the graph.\nQ: What is the maximum flow from node 2 to node 4?\nA:\n\n```\nmax_flow_from_node_2_to_node_4 = 12\n```\n'

In [155]:
print(instruction)
a = list(tokenizer.encode(instruction))
a

You are a good graph generator. You need to understand the task definition and generate a graph language to answer the question. 
Note: (i <-> j) means that node i and node j are connected with an undirected edge. (i -> j) means that node i and node j are connected with a directed edge. 
Task definition: given a graph structure description, generate a corresponding graph language. Note that: 1) the graph is an undirected-weighted graph and the name is "undirected-weighted-graph". 2) The generated graph language should be a code-like structure, and the skeleton format can be expressed as the following:
```
Graph[name="undirected-weighted-graph"] {
    entity_list = ["xxx", ...];
    triple_list = [("xxx" <-> "xxx")[weight="xxx"], ...];
}
```
Q: Given you a graph structure description, please generate a corresponding graph.
Description: "In an undirected graph, the nodes are numbered from 0 to 4, and the edges are:
an edge between node 0 and node 2 with weight 20,
an edge between node 0 

[1,
 887,
 526,
 263,
 1781,
 3983,
 15299,
 29889,
 887,
 817,
 304,
 2274,
 278,
 3414,
 5023,
 322,
 5706,
 263,
 3983,
 4086,
 304,
 1234,
 278,
 1139,
 29889,
 29871,
 13,
 9842,
 29901,
 313,
 29875,
 529,
 976,
 432,
 29897,
 2794,
 393,
 2943,
 474,
 322,
 2943,
 432,
 526,
 6631,
 411,
 385,
 563,
 1088,
 287,
 7636,
 29889,
 313,
 29875,
 1599,
 432,
 29897,
 2794,
 393,
 2943,
 474,
 322,
 2943,
 432,
 526,
 6631,
 411,
 263,
 10624,
 7636,
 29889,
 29871,
 13,
 5398,
 5023,
 29901,
 2183,
 263,
 3983,
 3829,
 6139,
 29892,
 5706,
 263,
 6590,
 3983,
 4086,
 29889,
 3940,
 393,
 29901,
 29871,
 29896,
 29897,
 278,
 3983,
 338,
 385,
 563,
 1088,
 287,
 29899,
 7915,
 287,
 3983,
 322,
 278,
 1024,
 338,
 376,
 870,
 1088,
 287,
 29899,
 7915,
 287,
 29899,
 4262,
 1642,
 29871,
 29906,
 29897,
 450,
 5759,
 3983,
 4086,
 881,
 367,
 263,
 775,
 29899,
 4561,
 3829,
 29892,
 322,
 278,
 18109,
 11285,
 3402,
 508,
 367,
 13384,
 408,
 278,
 1494,
 29901,
 13,
 28956,
 13,
 9

In [37]:
tokenizer.convert_ids_to_tokens([1, 319, 29901, 1576, 2714, 3983, 393, 4653, 267])

['<s>', '▁A', ':', 'The', '▁thought', '▁graph', '▁that', '▁express', 'es']

In [39]:
tokenizer.convert_tokens_to_ids("</s>")

2

In [160]:
tokenizer.eos_token_id

2

In [35]:
import copy
IGNORE_INDEX = -100
instruction = """A:"""

answer = """The thought graph that expresses the reasoning evidence is ```\nGraph[name="reasoning-thought-graph"] {\n    node_list = ["people", "complete job", "work", "job", "complete"];\n    edge_list = [("people" -> "complete job"[capacity="aim"]), ("work" -> "job"[capacity="place of take place"]), ("job" -> "complete"[capacity="goal"])];\n}\n```. \nThe answer should be the goal of people at work. Of the above choices, the goal of people at work is to complete the job. \nSo the answer is A. """


prompt = instruction
example = prompt + answer

prompt = tokenizer.encode(prompt)
example = tokenizer.encode(example) # list

if len(example) > 2048 - 1:
    example = example[:2047]

example.append(tokenizer.eos_token_id)
print(example)
prompt = torch.tensor(
    prompt, dtype=torch.int64
)

example = torch.tensor(
    example, dtype=torch.int64
)

labels = copy.deepcopy(example)
labels[: len(prompt)] = -1
example_mask = example.ge(0)
label_mask = labels.ge(0)
example[~example_mask] = 0
labels[~label_mask] = IGNORE_INDEX

[1, 319, 29901, 1576, 2714, 3983, 393, 4653, 267, 278, 24481, 10757, 338, 7521, 13, 9527, 29961, 978, 543, 23147, 292, 29899, 386, 1774, 29899, 4262, 3108, 426, 13, 1678, 2943, 29918, 1761, 353, 6796, 25719, 613, 376, 8835, 4982, 613, 376, 1287, 613, 376, 9057, 613, 376, 8835, 10370, 13, 1678, 7636, 29918, 1761, 353, 518, 703, 25719, 29908, 1599, 376, 8835, 4982, 29908, 29961, 5030, 5946, 543, 29874, 326, 3108, 511, 4852, 1287, 29908, 1599, 376, 9057, 29908, 29961, 5030, 5946, 543, 6689, 310, 2125, 2058, 3108, 511, 4852, 9057, 29908, 1599, 376, 8835, 29908, 29961, 5030, 5946, 543, 28111, 20068, 1385, 13, 29913, 13, 16159, 1412, 29871, 13, 1576, 1234, 881, 367, 278, 7306, 310, 2305, 472, 664, 29889, 4587, 278, 2038, 19995, 29892, 278, 7306, 310, 2305, 472, 664, 338, 304, 4866, 278, 4982, 29889, 29871, 13, 6295, 278, 1234, 338, 319, 29889, 29871, 2]


In [36]:
print(labels)

tensor([ -100,  -100,  -100,  1576,  2714,  3983,   393,  4653,   267,   278,
        24481, 10757,   338,  7521,    13,  9527, 29961,   978,   543, 23147,
          292, 29899,   386,  1774, 29899,  4262,  3108,   426,    13,  1678,
         2943, 29918,  1761,   353,  6796, 25719,   613,   376,  8835,  4982,
          613,   376,  1287,   613,   376,  9057,   613,   376,  8835, 10370,
           13,  1678,  7636, 29918,  1761,   353,   518,   703, 25719, 29908,
         1599,   376,  8835,  4982, 29908, 29961,  5030,  5946,   543, 29874,
          326,  3108,   511,  4852,  1287, 29908,  1599,   376,  9057, 29908,
        29961,  5030,  5946,   543,  6689,   310,  2125,  2058,  3108,   511,
         4852,  9057, 29908,  1599,   376,  8835, 29908, 29961,  5030,  5946,
          543, 28111, 20068,  1385,    13, 29913,    13, 16159,  1412, 29871,
           13,  1576,  1234,   881,   367,   278,  7306,   310,  2305,   472,
          664, 29889,  4587,   278,  2038, 19995, 29892,   278, 

In [1]:
[[1, 2, 3], [1, -2, 3], [4, 4, 5]]*3

[[1, 2, 3],
 [1, -2, 3],
 [4, 4, 5],
 [1, 2, 3],
 [1, -2, 3],
 [4, 4, 5],
 [1, 2, 3],
 [1, -2, 3],
 [4, 4, 5]]